In [1]:
!pip install -Uqq fastbook
!pip install patchify

In [2]:
import fastbook
fastbook.setup_book()
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/Kidney_stone_detection-CNN')
import KD_detector

In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import os
import random
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from keras.models import load_model
from tensorflow.keras.utils import plot_model

<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


In [4]:
imgs_path = r'/content/gdrive/My Drive/Colab Notebooks/Kidney_stone_detection-CNN/Dataset'

In [ ]:
train_images, train_labels = KD_detector.read_dataset(dataset='train', path=imgs_path, filter='none', size=-1)
len(train_images), len(train_labels)

/content/gdrive/My Drive/Colab Notebooks/Kidney_stone_detection-CNN/Dataset/Train/*
Kidney_stone


 62%|██████▏   | 484/781 [07:23<05:47,  1.17s/it]

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train_labels_encoded = le.fit_transform(train_labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_images, train_labels_encoded, test_size=0.1, random_state=42)

In [ ]:
X_train = X_train / 255.0

In [ ]:
y_train_onehot = to_categorical(y_train, num_classes=2)

In [ ]:
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)
print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)

In [ ]:
plt.imshow(X_train[1])

In [ ]:
from keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Activation
from tensorflow.keras.callbacks import ModelCheckpoint
from keras import regularizers

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(filters=16, kernel_size=(3,3), padding='valid', input_shape=(254, 254, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='valid'))

model.add(Conv2D(filters=32, kernel_size=(3,3), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='valid'))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='valid'))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='valid'))

model.add(Flatten())

model.add(Dense(units=150, kernel_regularizer=regularizers.l2(0.01), activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(units=100, kernel_regularizer=regularizers.l2(0.01), activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(units=2, activation='softmax'))

In [ ]:
#plot_model(model, to_file="model_topology.png", show_shapes=True, show_layer_activations=True)

In [ ]:
model.summary()

In [ ]:
#plot_model(model, to_file="model_topology.png")

In [ ]:
model_filepath = "Model/model-{epoch:02d}-{val_accuracy:.4f}.h5"

In [ ]:
checkpoint = ModelCheckpoint(filepath=model_filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train_onehot, epochs=100, batch_size=32, validation_split=0.1, callbacks=[checkpoint])

In [ ]:
X_test = X_test / 255.0

In [ ]:
y_test_onehot = to_categorical(y_test, num_classes=2)

In [ ]:
model.evaluate(X_test, y_test_onehot, verbose=1)

In [ ]:
prediction = model.predict(X_test)

In [ ]:
y_pred = np.argmax(prediction, axis=1)

In [ ]:
print('Predicted: ', y_pred[0:30])
print('Actual:    ', y_test[0:30])

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
ax.plot(history.history['accuracy'])
ax.plot(history.history['val_accuracy'])
ax.set_title('Model accuracy')
ax.set_ylabel('accuracy')
ax.set_xlabel('epoch')
ax.legend(['train', 'val'], loc='upper left')

In [ ]:
target_names = ['KD', 'Normal']
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)

In [ ]:
print(cm)

In [ ]:
def plot_cm(cm, classes, normalize=False, title='Confusion Matrix', cmap=plt.cm.Blues):
    fig, ax = plt.subplots(figsize=(6,6))
    ax = sns.heatmap(cm, annot=True, linewidths=.5, ax=ax)

    ax.set_title(title)
    tick_marks = np.arange(len(classes))
    tick_marks = [0.5, 1.5]
    print(tick_marks)
    ax.set_xticks(tick_marks)
    ax.set_xticklabels(classes)
    ax.set_yticks(tick_marks)
    ax.set_yticklabels(classes)

    plt.tight_layout()
    ax.set_ylabel('True label')
    ax.set_xlabel('Predicted label')

In [ ]:
cm_plot_labels = ['KD', 'Normal']

In [ ]:
plot_cm(cm=cm, classes=cm_plot_labels, title='Confustion Matrix')

In [ ]:
print(y_pred[6])

In [ ]:
def plot_images(X_test, y_pred, y_test, grid_size):
    font_style = {'fontsize': 12, 'fontweight': 2}
    rows = grid_size[0]
    cols = grid_size[1]
    panel = rows*cols
    fig = plt.figure(figsize=(20, 20))  # width, height in inches
    for i in range(0, panel):
        ax = fig.add_subplot(rows, cols, i + 1)

        if y_pred[i] == 1 and y_pred[i] == y_test[i]:
            kd_percent = np.min(prediction[i]) * 100
            nm_percent = np.max(prediction[i]) * 100
            color = 'green'
        elif y_pred[i] == 0 and y_pred[i] == y_test[i]:
            kd_percent = np.max(prediction[i]) * 100
            nm_percent = np.min(prediction[i]) * 100
            color = 'green'
        elif y_pred[i] == 1 and y_pred[i] != y_test[i]:
            kd_percent = np.min(prediction[i]) * 100
            nm_percent = np.max(prediction[i]) * 100
            color = 'red'
        elif y_pred[i] == 0 and y_pred[i] != y_test[i]:
            kd_percent = np.max(prediction[i]) * 100
            nm_percent = np.min(prediction[i]) * 100
            color = 'red'

        ax.set_title(f"Kidney_stone: {kd_percent:.2f}%\nNormal: {nm_percent:.2f}%", fontdict=font_style, color=color)

        ax.imshow(X_test[i], interpolation='nearest', cmap='gray')

    fig.tight_layout()

In [ ]:
plot_images(X_test, y_pred, y_test, grid_size=(10,10))